In [ ]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config


## Read data into pandas dataframe

How do we define which files should be stitched together?
options:
- all files in folder
- based on suffix (e.g. "n=895_06-Feb-2023_PVC2.tsv")
- check first columns to see whether it matches

For now, we will will use the first option

In [ ]:
# Identify files
experiment_folder = 'TOP'

config = Config()
root_directory = config.get_directory('raw_data')
tsv_path = os.path.join(root_directory, experiment_folder)

tsv_files = [os.path.join(tsv_path, file) for file in os.listdir(tsv_path) if file.endswith('.tsv')]
tsv_files

In [ ]:
# Read files into dataframes
dataframes = [pd.read_csv(file, sep='\t', header=[0,1], index_col=0) for file in tsv_files]
sample_df = dataframes[0]
cols = sample_df.columns
sample_df

## Copy identical columns from any file

we could also read it from the data, but if it's always the same, we can just define it here

In [ ]:
# how many identical columns are there in the files?
n_identical = 11  # columns A - L

stitched = sample_df[cols[:n_identical]].copy()
stitched


In [ ]:
# Would be nice to add a test here to double check that identical columns are actually indentical

## Add unique columns from files

In [ ]:
for df in dataframes:
    for col in df.columns[n_identical:]:
        stitched[col] = df[col]

stitched

## Add sex and age data

In [ ]:
sexage_path = os.path.join(root_directory, 'age_data', 'Age_Sex_TOP.csv')
sexage_df = pd.read_csv(sexage_path, index_col=0)
sexage_df




In [ ]:
for col in sexage_df[2:]:
    stitched[col] = sexage_df[col]

stitched[['GM_vol', 'Sex', 'Age']]

## save off file

In [ ]:
 
# filepath = '../open_work/internal_results/top_stitched.csv' 
# stitched.to_csv(filepath)  

# Look at columns

In [ ]:
stitched.columns

In [ ]:
stitched

In [ ]:
def concat_double_header(dataframe_dub):
    dataframe = dataframe_dub.copy()
    dataframe.columns = [c[0] + "_" + c[1] for c in dataframe.columns]
    return dataframe

In [ ]:
lo = concat_double_header(stitched)
lo.columns

In [ ]:


dataframe = stitched.copy()
dataframe.columns = [c[0] + "_" + c[1] for c in dataframe.columns]

#dataframe.columns

In [ ]:
# col=dataframe.columns.to_list()

# for i in col:
#     print(dataframe[i].shape)
#     print(dataframe[i])

In [ ]:
# special_column = 'Age'
# all_columns = dataframe.columns
# notmain = dataframe.drop(special_column, axis=1)
# notmain_columns = notmain.columns

In [ ]:
def relate_columns_graphs(dataframe, special_column_name):
    y = dataframe[special_column_name]
    col = dataframe.columns.to_list()
    a = len(col)  # number of rows
    b = 1  # number of columns
    c = 1  # initialize plot counter
    
    fig = plt.figure(figsize=(10,(len(col)*3)))
    for i in col:
        plt.subplot(a, b, c)
        plt.scatter(dataframe[i], y)
        plt.title('{}, subplot: {}{}{}'.format(i, a, b, c))
        plt.xlabel(i)
        c = c + 1
    plt.savefig(("versus"+special_column_name +".png")) 
                     


In [ ]:
sep.relate_columns_graphs(dataframe, 'Age_')